# Jupyter notebook for taking the BLASTx output (crab BLAST output : `anti_join` of dinoflagellate BLAST output and uniprot BLAST output) to GOslim


### from Steven's notebook: https://github.com/sr320/nb-2018/blob/master/C_virginica/83-blast-2-slim.ipynb

### Need three files:
1) blastout file in format -6 (https://raw.githubusercontent.com/RobertsLab/project-crab/master/analyses/091419-crab-blastx-output.csv)       
2) Uniprot GO annotation file (340M) from : http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/uniprot-SP-GO.sorted       
3) GOslim file from : http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/Go-GOslim.sorted

In [1]:
pwd

'/Users/graciecrandall/Documents/GitHub/project-crab/notebooks'

In [2]:
wd = "/Users/graciecrandall/Documents/GitHub/project-crab/analyses/"

In [3]:
cd {wd}

/Users/graciecrandall/Documents/GitHub/project-crab/analyses


In [4]:
!curl -O http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/GO-GOslim.sorted

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2314k  100 2314k    0     0  2840k      0 --:--:-- --:--:-- --:--:-- 3024k


In [5]:
!curl -O http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/uniprot-SP-GO.sorted 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  340M  100  340M    0     0  9931k      0  0:00:35  0:00:35 --:--:-- 18.9M


In [22]:
# check if files are in the directory
!ls

091419-crab-blastx-output.csv     _intermediate.file
1108-cbairdiblast-sprot.tab       crab-GOslim.csv
Blastquery-GOslim.tab             crab-blastx.tab
Cb_v1_blastx_uniprot2864_imac.tab dino-Blastquery-GOslim.sorted
GO-GOslim.sorted                  hemat-Blastquery-GOslim.sorted
_blast-GO-unfolded-nos.sorted     hemat-blastx-output.tab
_blast-GO-unfolded.sorted         master-qubit-sample.csv
_blast-GO-unfolded.tab            master-qubit.xlsx
_blast-annot.tab                  qubit_results-for-libraries.csv
_blast-sep.tab                    readme.md
_blast-sep2.tab                   sample_table.csv
_blast-sep3.tab                   uniprot-SP-GO.sorted
_blast-sort.tab


### Set blastout as a variable

In [23]:
blastout="091419-crab-blastx-output.csv"

In [24]:
!head -2 $blastout

In [25]:
!wc -l $blastout

  114378 091419-crab-blastx-output.csv


In [26]:
# convert pipes to tab
!tr '|' '\t' < $blastout \
> _blast-sep.tab

In [27]:
!head -2 _blast-sep.tab

In [28]:
#convert commas to tab
!tr ',' '\t' < _blast-sep.tab \
> _blast-sep2.tab

In [29]:
!head -2 _blast-sep2.tab

In [31]:
#reducing number of columns and sorting 
!awk -v OFS='\t' '{print $3, $1, $13}' <_blast-sep2.tab | sort \
> _blast-sort.tab

In [32]:
!head -2 _blast-sort.tab

A0A023PXK7	TRINITY_DN19072_c0_g3_i2	4
A0A023PXK7	TRINITY_DN8794_c0_g2_i1	2.3


In [33]:
#joining blast with uniprot annotation file and reducing to three columns UniprotID, Query, All GO terms
!join -t $'\t' \
_blast-sort.tab \
uniprot-SP-GO.sorted \
| cut -f1,2,14 \
> _blast-annot.tab

In [34]:
!head _blast-annot.tab

A0A023PXK7	TRINITY_DN19072_c0_g3_i2	GO:0016021
A0A023PXK7	TRINITY_DN8794_c0_g2_i1	GO:0016021
A0A023PYE4	TRINITY_DN24825_c0_g1_i1	GO:0016021
A0A023PYH8	TRINITY_DN40894_c0_g1_i1	
A0A023PYJ0	TRINITY_DN14110_c0_g1_i2	GO:0016021
A0A023PYJ7	TRINITY_DN6907_c0_g3_i2	GO:0016021
A0A023PYJ7	TRINITY_DN6907_c0_g3_i3	GO:0016021
A0A023PYK2	TRINITY_DN15377_c0_g2_i2	GO:0016021
A0A023PZ99	TRINITY_DN25724_c0_g2_i1	GO:0016021
A0A023PZA4	TRINITY_DN11962_c0_g1_i2	


### The following is a script modified by Sam White

In [35]:
%%bash 

# This script was originally written to address a specific problem that Rhonda was having



# input_file is the initial, "problem" file
# file is an intermediate file that most of the program works upon
# output_file is the final file produced by the script
input_file="_blast-annot.tab"
file="_intermediate.file"
output_file="_blast-GO-unfolded.tab"

# sed command substitutes the "; " sequence to a tab and writes the new format to a new file.
# This character sequence is how the GO terms are delimited in their field.
sed $'s/; /\t/g' "$input_file" > "$file"

# Identify first field containing a GO term.
# Search file with grep for "GO:" and pipe to awk.
# Awk sets tab as field delimiter (-F'\t'), runs a for loop that looks for "GO:" (~/GO:/), and then prints the field number).
# Awk results are piped to sort, which sorts unique by number (-ug).
# Sort results are piped to head to retrieve the lowest value (i.e. the top of the list; "-n1").
begin_goterms=$(grep "GO:" "$file" | awk -F'\t' '{for (i=1;i<=NF;i++) if($i ~/GO:/) print i}' | sort -ug | head -n1)

# While loop to process each line of the input file.
while read -r line
	do
	
	# Send contents of the current line to awk.
	# Set the field separator as a tab (-F'\t') and print the number of fields in that line.
	# Save the results of the echo/awk pipe (i.e. number of fields) to the variable "max_field".
	max_field=$(echo "$line" | awk -F'\t' '{print NF}')

	# Send contents of current line to cut.
	# Cut fields (i.e. retain those fields) 1-12.
	# Save the results of the echo/cut pipe (i.e. fields 1-12) to the variable "fixed_fields"
	fixed_fields=$(echo "$line" | cut -f1-2)

	# Since not all the lines contain the same number of fields (e.g. may not have GO terms),
	# evaluate the number of fields in each line to determine how to handle current line.

	# If the value in max_field is less than the field number where the GO terms begin,
	# then just print the current line (%s) followed by a newline (\n).
	if (( "$max_field" < "$begin_goterms" ))
		then printf "%s\n" "$line"
			else

			# Send contents of current line (which contains GO terms) to cut.
			# Cut fields (i.e. retain those fields) 13 to whatever the last field is in the curent line.
			# Save the results of the echo/cut pipe (i.e. all the GO terms fields) to the variable "goterms".
			goterms=$(echo "$line" | cut -f"$begin_goterms"-"$max_field")
			
			# Assign values in the variable "goterms" to a new indexed array (called "array"), 
			# with tab delimiter (IFS=$'\t')
			IFS=$'\t' read -r -a array <<<"$goterms"
			
			# Iterate through each element of the array.
			# Print the first 12 fields (i.e. the fields stored in "fixed_fields") followed by a tab (%s\t).
			# Print the current element in the array (i.e. the current GO term) followed by a new line (%s\n).
			for element in "${!array[@]}"	
				do printf "%s\t%s\n" "$fixed_fields" "${array[$element]}"
			done
	fi

# Send the input file into the while loop and send the output to a file named "rhonda_fixed.txt".
done < "$file" > "$output_file"

Process is interrupted.


In [36]:
!head _blast-GO-unfolded.tab

A0A023PXK7	TRINITY_DN19072_c0_g3_i2	GO:0016021
A0A023PXK7	TRINITY_DN8794_c0_g2_i1	GO:0016021
A0A023PYE4	TRINITY_DN24825_c0_g1_i1	GO:0016021
A0A023PYH8	TRINITY_DN40894_c0_g1_i1
A0A023PYJ0	TRINITY_DN14110_c0_g1_i2	GO:0016021
A0A023PYJ7	TRINITY_DN6907_c0_g3_i2	GO:0016021
A0A023PYJ7	TRINITY_DN6907_c0_g3_i3	GO:0016021
A0A023PYK2	TRINITY_DN15377_c0_g2_i2	GO:0016021
A0A023PZ99	TRINITY_DN25724_c0_g2_i1	GO:0016021
A0A023PZA4	TRINITY_DN11962_c0_g1_i2


In [37]:
!awk '{print $3,"\t" $2}' _blast-GO-unfolded.tab | gsort -V > _blast-GO-unfolded.sorted

In [38]:
!head _blast-GO-unfolded.sorted

GO:0000001 	TRINITY_DN24641_c0_g1_i1
GO:0000001 	TRINITY_DN44029_c0_g1_i1
GO:0000002 	TRINITY_DN654_c0_g1_i3
GO:0000002 	TRINITY_DN7877_c0_g2_i2
GO:0000002 	TRINITY_DN10626_c0_g3_i1
GO:0000002 	TRINITY_DN10677_c0_g1_i4
GO:0000002 	TRINITY_DN14406_c0_g3_i1
GO:0000002 	TRINITY_DN15295_c0_g2_i1
GO:0000002 	TRINITY_DN15295_c0_g2_i10
GO:0000002 	TRINITY_DN15473_c1_g2_i1


In [39]:
!tr " " "*" < _blast-GO-unfolded.sorted | head

GO:0000001*	TRINITY_DN24641_c0_g1_i1
GO:0000001*	TRINITY_DN44029_c0_g1_i1
GO:0000002*	TRINITY_DN654_c0_g1_i3
GO:0000002*	TRINITY_DN7877_c0_g2_i2
GO:0000002*	TRINITY_DN10626_c0_g3_i1
GO:0000002*	TRINITY_DN10677_c0_g1_i4
GO:0000002*	TRINITY_DN14406_c0_g3_i1
GO:0000002*	TRINITY_DN15295_c0_g2_i1
GO:0000002*	TRINITY_DN15295_c0_g2_i10
GO:0000002*	TRINITY_DN15473_c1_g2_i1


In [40]:
!sed 's/ //g' _blast-GO-unfolded.sorted > _blast-GO-unfolded-nos.sorted

In [41]:
!head _blast-GO-unfolded-nos.sorted

GO:0000001	TRINITY_DN24641_c0_g1_i1
GO:0000001	TRINITY_DN44029_c0_g1_i1
GO:0000002	TRINITY_DN654_c0_g1_i3
GO:0000002	TRINITY_DN7877_c0_g2_i2
GO:0000002	TRINITY_DN10626_c0_g3_i1
GO:0000002	TRINITY_DN10677_c0_g1_i4
GO:0000002	TRINITY_DN14406_c0_g3_i1
GO:0000002	TRINITY_DN15295_c0_g2_i1
GO:0000002	TRINITY_DN15295_c0_g2_i10
GO:0000002	TRINITY_DN15473_c1_g2_i1


In [42]:
!tr " " "*" < _blast-GO-unfolded-nos.sorted | head

GO:0000001	TRINITY_DN24641_c0_g1_i1
GO:0000001	TRINITY_DN44029_c0_g1_i1
GO:0000002	TRINITY_DN654_c0_g1_i3
GO:0000002	TRINITY_DN7877_c0_g2_i2
GO:0000002	TRINITY_DN10626_c0_g3_i1
GO:0000002	TRINITY_DN10677_c0_g1_i4
GO:0000002	TRINITY_DN14406_c0_g3_i1
GO:0000002	TRINITY_DN15295_c0_g2_i1
GO:0000002	TRINITY_DN15295_c0_g2_i10
GO:0000002	TRINITY_DN15473_c1_g2_i1


In [43]:
!head GO-GOslim.sorted

GO:0000001	mitochondrion inheritance	cell organization and biogenesis	P
GO:0000002	mitochondrial genome maintenance	cell organization and biogenesis	P
GO:0000003	reproduction	other biological processes	P
GO:0000006	high affinity zinc uptake transmembrane transporter activity	transporter activity	F
GO:0000007	low-affinity zinc ion transmembrane transporter activity	transporter activity	F
GO:0000009	"alpha-1,6-mannosyltransferase activity"	other molecular function	F
GO:0000010	trans-hexaprenyltranstransferase activity	other molecular function	F
GO:0000011	vacuole inheritance	cell organization and biogenesis	P
GO:0000012	single strand break repair	DNA metabolism	P
GO:0000012	single strand break repair	stress response	P


In [44]:
!join -1 1 -2 1 -t $'\t' \
_blast-GO-unfolded-nos.sorted \
GO-GOslim.sorted | head 

GO:0000001	TRINITY_DN24641_c0_g1_i1	mitochondrion inheritance	cell organization and biogenesis	P
GO:0000001	TRINITY_DN44029_c0_g1_i1	mitochondrion inheritance	cell organization and biogenesis	P
GO:0000002	TRINITY_DN654_c0_g1_i3	mitochondrial genome maintenance	cell organization and biogenesis	P
GO:0000002	TRINITY_DN7877_c0_g2_i2	mitochondrial genome maintenance	cell organization and biogenesis	P
GO:0000002	TRINITY_DN10626_c0_g3_i1	mitochondrial genome maintenance	cell organization and biogenesis	P
GO:0000002	TRINITY_DN10677_c0_g1_i4	mitochondrial genome maintenance	cell organization and biogenesis	P
GO:0000002	TRINITY_DN14406_c0_g3_i1	mitochondrial genome maintenance	cell organization and biogenesis	P
GO:0000002	TRINITY_DN15295_c0_g2_i1	mitochondrial genome maintenance	cell organization and biogenesis	P
GO:0000002	TRINITY_DN15295_c0_g2_i10	mitochondrial genome maintenance	cell organization and biogenesis	P
GO:0000002	TRINITY_DN15473_c1_g2_i1	mitochondrial genome maintenance	cell organi

In [45]:
#joining files to get GOslim for each query (with duplicate GOslim / query removed)
!join -1 1 -2 1 -t $'\t' \
_blast-GO-unfolded-nos.sorted \
GO-GOslim.sorted \
| uniq | awk -F'\t' -v OFS='\t' '{print $2, $4, $5}' \
> Blastquery-GOslim.tab
!wc -l Blastquery-GOslim.tab

   82623 Blastquery-GOslim.tab


In [46]:
!head Blastquery-GOslim.tab
!wc -l Blastquery-GOslim.tab

TRINITY_DN24641_c0_g1_i1	cell organization and biogenesis	P
TRINITY_DN44029_c0_g1_i1	cell organization and biogenesis	P
TRINITY_DN654_c0_g1_i3	cell organization and biogenesis	P
TRINITY_DN7877_c0_g2_i2	cell organization and biogenesis	P
TRINITY_DN10626_c0_g3_i1	cell organization and biogenesis	P
TRINITY_DN10677_c0_g1_i4	cell organization and biogenesis	P
TRINITY_DN14406_c0_g3_i1	cell organization and biogenesis	P
TRINITY_DN15295_c0_g2_i1	cell organization and biogenesis	P
TRINITY_DN15295_c0_g2_i10	cell organization and biogenesis	P
TRINITY_DN15473_c1_g2_i1	cell organization and biogenesis	P
   82623 Blastquery-GOslim.tab


### Get -P query slim

In [48]:
!awk -F"\t" '$3 == "F" { print $1"\t"$2 }' Blastquery-GOslim.tab | sort > crab-Blastquery-GOslim.sorted
!head crab-Blastquery-GOslim.sorted

TRINITY_DN1000_c0_g1_i4	other molecular function
TRINITY_DN10014_c0_g1_i1	other molecular function
TRINITY_DN10014_c0_g1_i1	other molecular function
TRINITY_DN10015_c0_g1_i1	nucleic acid binding activity
TRINITY_DN10027_c1_g1_i1	other molecular function
TRINITY_DN10027_c1_g1_i1	other molecular function
TRINITY_DN10027_c1_g1_i1	other molecular function
TRINITY_DN10031_c0_g2_i1	other molecular function
TRINITY_DN10035_c0_g1_i2	kinase activity
TRINITY_DN10035_c0_g1_i2	other molecular function
